In [1]:
import pandas as pd

In [4]:
access = pd.read_csv("/home/jaime/projet_info_3A/private/access.csv",header=0)
id_api = access["Identifiant"][0]
key_api = access["Clé secrète"][0]

In [5]:
import requests 
import os

In [6]:
import emploi_store

In [7]:
client_id = id_api
client_secret = key_api
access_tokens = {}
cached_packages = None
scope = 'api_offresdemploiv2'

In [8]:
# from https://github.com/Cocorico84/employme/blob/855cfcda2c54f42b9c42dcbf4ac87cdf09248b7f/back/manager.py

In [9]:
import requests


url = "https://api.emploi-store.fr/partenaire"

In [10]:
def get_token():
    url_token = 'https://entreprise.pole-emploi.fr/connexion/oauth2/access_token'
    r = requests.post(url_token, headers={'Content-Type': 'application/x-www-form-urlencoded'},
                      params={'realm': '/partenaire'},
                      data={'grant_type': 'client_credentials', 'client_id': id_api,
                            'client_secret': key_api,
                            'scope': f'application_{id_api} api_offresdemploiv2 o2dsoffre'})
    data = r.json()
    return data['access_token']


In [11]:
def connection(token):
    url_offers = f'{url}/offresdemploi/v2/offres/search'
    r = requests.get(url=url_offers, #,  params={'motsCles': 'informatique'}
        headers={'Authorization': f'Bearer {get_token()}'})
    data = r.json()

    return data

In [12]:
request_as_dict=connection(get_token())

In [27]:
df = pd.DataFrame.from_dict(request_as_dict['resultats'])


In [28]:
df = pd.concat([df.drop(['lieuTravail'], axis=1), df['lieuTravail'].apply(pd.Series)], axis=1)

df = pd.concat([df.drop(['origineOffre'], axis=1), df['origineOffre'].apply(pd.Series)], axis=1)
df = pd.concat([df.drop(['partenaires'], axis=1), df['partenaires'].apply(pd.Series)], axis=1)


In [31]:
df.to_csv("df_clean_une_apge.csv", sep=",", header=False, index=False)